In [60]:
import csv
from copy import copy

import openpyxl
from openpyxl import load_workbook
from openpyxl.formatting import Rule
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
from openpyxl.worksheet.cell_range import CellRange
from openpyxl.utils import coordinate_from_string, column_index_from_string

import pandas as pd

In [80]:
wb = openpyxl.Workbook()
ws = wb.active

entries_csv = r"C:\Users\aaron\dev\dfs\DKEntries.csv"

with open(entries_csv) as f:
    reader = csv.reader(f)
    for row in reader:
        ws.append(row)

wb.save(r"C:\Users\aaron\dev\dfs\DKEntries.xlsx")

In [81]:
filename = r"C:\Users\aaron\dev\dfs\DKEntries.xlsx"
wb = load_workbook(filename)
ws = wb.active

In [82]:
end_col_row_coord = ws.dimensions.split(':')[1]
xy = coordinate_from_string(end_col_row_coord)
end_col = column_index_from_string(xy[0])
end_row = xy[1]

print(end_col_row_coord)
print(xy)
print(end_col)
print(end_row)

U58
('U', 58)
21
58


In [83]:
colors_csv = r"C:\Users\aaron\dev\dfs\NHL_Team_Colors.csv"
colors = pd.read_csv(colors_csv, dtype='str', index_col=False)
colors

,Team,Font Color,Fill Color
0,ANH,110000,EF5225
1,ARI,EFE1C6,841F27
2,BOS,110000,FFC422
3,BUF,FDBB2F,002E62
4,CGY,FFC758,E03A3E
5,CAR,E03A3E,8E8E90
6,CHI,110000,E3263A
7,COL,A9B0B8,8B2942
8,CLS,E03A3E,00285C
9,DAL,C0C0C0,006A4E


In [84]:
col_dict = colors.set_index('Team').T.to_dict('list')
col_dict

{'ANH': ['110000', 'EF5225'],
 'ARI': ['EFE1C6', '841F27'],
 'BOS': ['110000', 'FFC422'],
 'BUF': ['FDBB2F', '002E62'],
 'CGY': ['FFC758', 'E03A3E'],
 'CAR': ['E03A3E', '8E8E90'],
 'CHI': ['110000', 'E3263A'],
 'COL': ['A9B0B8', '8B2942'],
 'CLS': ['E03A3E', '00285C'],
 'DAL': ['C0C0C0', '006A4E'],
 'DET': ['FFFFFF', 'EC1F26'],
 'EDM': ['E66A20', '003777'],
 'FLA': ['B9975B', '002E5F'],
 'LAK': ['AFB7BA', '110000'],
 'MIN': ['EFB410', '025736'],
 'MON': ['213770', 'BF2F38'],
 'NSH': ['002E62', 'FDBB2F'],
 'NJ': ['E03A3E', '110000'],
 'NYI': ['F57D31', '00529B'],
 'NYR': ['E6393F', '0161AB'],
 'OTT': ['E4173E', 'D69F0F'],
 'PHI': ['110000', 'F47940'],
 'PIT': ['110000', 'D1BD80'],
 'SJ': ['F35F20', '05535D'],
 'STL': ['0546A0', 'FFC325'],
 'TB': ['C0C0C0', '013E7D'],
 'TOR': ['FFFFFF', '003777'],
 'VAN': ['047A4A', '07346F'],
 'WAS': ['00214E', 'CF132B'],
 'WPG': ['002E62', 'A8A9AD'],
 'VGK': ['B4975A', '333F42']}

In [85]:
for key, value in col_dict.items():
    text = Font(color=value[0])
    fill = PatternFill(bgColor=value[1])
    dxf = DifferentialStyle(font=text, fill=fill)
    rule = Rule(type="containsText", operator="containsText", text=key, dxf=dxf)
    rule.formula = ['=$T9="{}"'.format(key)]
    ws.conditional_formatting.add('N9:{}'.format(end_col_row_coord), rule)
    wb.save(filename)

## Copy 'Name+ID' column, paste into MS Word, then copy back to same column in spreadsheet

In [8]:
#Copy range of cells as a nested list
#Takes: start cell, end cell, and sheet you want to copy from.

#Only copies cell values.  Experiment with copy/deepcopy to grab formatting as well.
def copyRange(startCol, startRow, endCol, endRow, sheet):
    rangeSelected = []
    #Loops through selected Rows
    for i in range(startRow,endRow + 1,1):
        #Appends the row to a RowSelected list
        rowSelected = []
        for j in range(startCol,endCol+1,1):
            rowSelected.append(sheet.cell(row = i, column = j).value)
        #Adds the RowSelected List and nests inside the rangeSelected
        rangeSelected.append(rowSelected)
 
    return rangeSelected

In [9]:
#Paste range
#Paste data from copyRange into template sheet
def pasteRange(startCol, startRow, endCol, endRow, sheetReceiving,copiedData):
    countRow = 0
    for i in range(startRow,endRow+1,1):
        countCol = 0
        for j in range(startCol,endCol+1,1):
            
            sheetReceiving.cell(row = i, column = j).value = copiedData
            countCol += 1
        countRow += 1

In [10]:
def createData():
    print("Processing...")
    selectedRange = copyRange(1,2,4,14,sheet) #Change the 4 number values
    pastingRange = pasteRange(1,3,4,15,temp_sheet,selectedRange) #Change the 4 number values
    #You can save the template as another file to create a new file here too.s
    template.save("foo.xlsx")
    print("Range copied and pasted!")

In [86]:
player_pool = copyRange(startCol=14, startRow=9, endCol=end_col, endRow=end_row, sheet=ws)

for player in player_pool:
    if (player[4]) == 'T1':
        print(player[1])

Alex Ovechkin (12118157)
Brent Burns (12118158)
Sidney Crosby (12118159)
Nikita Kucherov (12118160)


In [13]:
player_pool = copyRange(startCol=14, startRow=9, endCol=end_col, endRow=end_row, sheet=ws)
player_pool

[['C',
  'Jack Eichel (12101668)',
  'Jack Eichel',
  '12101668',
  'T1',
  'BUF@NJ 02/17/2019 06:00PM ET',
  'BUF',
  '4.86'],
 ['C',
  'Dylan Larkin (12101667)',
  'Dylan Larkin',
  '12101667',
  'T1',
  'PHI@DET 02/17/2019 06:00PM ET',
  'DET',
  '4.54'],
 ['LW',
  'Alex Ovechkin (12101669)',
  'Alex Ovechkin',
  '12101669',
  'T1',
  'WAS@ANH 02/17/2019 09:00PM ET',
  'WAS',
  '5.34'],
 ['C',
  'Evgeny Kuznetsov (12101670)',
  'Evgeny Kuznetsov',
  '12101670',
  'T1',
  'WAS@ANH 02/17/2019 09:00PM ET',
  'WAS',
  '3.94'],
 ['LW',
  'Jeff Skinner (12101672)',
  'Jeff Skinner',
  '12101672',
  'T2',
  'BUF@NJ 02/17/2019 06:00PM ET',
  'BUF',
  '4.32'],
 ['C',
  'Sean Couturier (12101671)',
  'Sean Couturier',
  '12101671',
  'T2',
  'PHI@DET 02/17/2019 06:00PM ET',
  'PHI',
  '4.10'],
 ['C',
  'Aleksander Barkov (12101673)',
  'Aleksander Barkov',
  '12101673',
  'T2',
  'MON@FLA 02/17/2019 07:00PM ET',
  'FLA',
  '3.91'],
 ['LW',
  'Jonathan Drouin (12101674)',
  'Jonathan Drouin',
